In [1]:
from saleor.checkout.utils import *
from saleor.product.utils import *
from saleor.payment.utils import *
from saleor.payment.gateway import *

### Fannying around with address forms

In [1]:
from i18naddress import normalize_address, InvalidAddress

In [2]:
address = normalize_address({
    'country_code': 'US',
    'country_area': 'California',
    'city': 'Mountain View',
    'postal_code': '94043',
    'street_address': '1600 Amphitheatre Pkwy'})

In [3]:
address

{'country_code': 'US',
 'country_area': 'CA',
 'city': 'MOUNTAIN VIEW',
 'postal_code': '94043',
 'street_address': '1600 Amphitheatre Pkwy',
 'city_area': '',
 'sorting_code': ''}

In [4]:
try:
    address = normalize_address({'country_code': 'US'})
except InvalidAddress as e:
    print(e.errors)

{'country_area': 'required', 'city': 'required', 'postal_code': 'required', 'street_address': 'required'}


In [4]:
%pdb

Automatic pdb calling has been turned ON


In [1]:
from saleor.account import i18n

In [34]:
d = {'csrfmiddlewaretoken': ['cxHqQBwfi6Dt6hhXpyHEdsoWdAY5AxMk7lmMFLYQiWcYpQOotkhQEscuHb3HyIda', 'cxHqQBwfi6Dt6hhXpyHEdsoWdAY5AxMk7lmMFLYQiWcYpQOotkhQEscuHb3HyIda'], 'address': ['new_address'], 'first_name': ['Christopher'], 'last_name': ['Kelly'], 'company_name': ['Nourish Balance Thrive'], 'street_address_1': ['471 Country Estates Ter'], 'street_address_2': [''], 'city': ['Santa Cruz'], 'country_area': ['CA'], 'postal_code': ['95060'], 'city_area': [''], 'country': ['US'], 'phone_1': [''], 'preview': [''], 'shipping_method': ['13'], 'cardholder-name': ['Christopher Kelly'], 'payment_method_id': ['pm_1FmLG5DGt0TthQJwmdlFHvWr']}

In [35]:
form = i18n.COUNTRY_FORMS['US'](d)

In [99]:
form = i18n.COUNTRY_FORMS['US']()

In [101]:
form

<AddressFormUS bound=False, valid=False, fields=(first_name;last_name;company_name;street_address_1;street_address_2;city;city_area;postal_code;country;country_area;phone)>

In [102]:
form.is_valid(), form.errors

(False, {})

In [103]:
form._errors

{}

In [6]:
[e.message for e in form.errors.as_data().get('country')]

['This field is required.']

In [21]:
i18naddress.__file__

'/home/cck197/saleor/venv2/lib/python3.7/site-packages/i18naddress/__init__.py'

In [104]:
i18n.__file__

'/home/cck197/saleor/saleor/account/i18n.py'

### ProductVariant

In [18]:
p = Product.objects.get(id=121)

In [19]:
[str(v) for v in p.get_attr_startswith('ppf_')]

['no subscription', 'no trials', 'one-time payment']

In [20]:
p.price

Money('34.99', 'USD')

In [26]:
p.images.filter(image__name='products/EKC-3rd-3d-s.png')

FieldError: Unsupported lookup 'name' for VersatileImageField or join on the field not permitted.

In [48]:
[i.url for i in p.images.all()]

AttributeError: 'ProductImage' object has no attribute 'url'

In [40]:
def get_product_image(product, name):
    images = [i for i in product.images.all() if i.image.name == name]
    if len(images):
        return images[0]
    else:
        raise Exception(f"VersatileImageField {name} not found")

In [58]:
def get_product_image(product, alt):
    try:
        return product.images.filter(alt=alt).first().image.url
    except AttributeError:
        return None

In [59]:
get_image_by_alt(p, 'foo')

In [42]:
get_product_image(p, 'products/EKC-3d-image-2-min-1_v3qDvl7.jpg')

<ProductImage: ProductImage object (56)>

In [43]:
p.__dict__

{'_state': <django.db.models.base.ModelState at 0x7fed2aae71d0>,
 'id': 121,
 'publication_date': None,
 'is_published': True,
 'private_meta': {},
 'meta': {},
 'seo_title': '',
 'seo_description': 'With 105 mouth-watering recipes in The Essential Keto Cookbook, you’ve got meals, snacks, desserts, and breakfasts your entire family can enjoy for any occasion.',
 'product_type_id': 16,
 'name': 'Essential Keto Cookbook',
 'description': '<p>With 105 mouth-watering recipes in The Essential Keto Cookbook, you’ve got meals, snacks, desserts, and breakfasts your entire family can enjoy for any occasion.</p>',
 'description_json': {},
 'category_id': 10,
 'currency': 'USD',
 'price_amount': Decimal('34.99'),
 'minimal_variant_price_amount': Decimal('0.00'),
 'updated_at': datetime.datetime(2020, 1, 2, 18, 1, 30, 993366, tzinfo=<UTC>),
 'charge_taxes': True,
 'weight': None}

In [387]:
[a.attribute.slug for a in p.attributes.all()]#(attribute__slug='pf_1')]

['funnel_order', 'pf_1', 'pf_2', 'pf_3', 'pf_4']

In [390]:
for attribute_rel in p.attributes.all():
    attribute = attribute_rel.attribute
    if 'pf_' in attribute.slug:
        print(f'{attribute.translated} ({attribute.slug}) = ', end='')
        for value in attribute_rel.values.all():
            print(value.translated)

Product Features 1 (pf_1) = 104+ Keto Recipes with Full Nutritional Data
Product Features 2 (pf_2) = Full Color Physical Cookbook
Product Features 3 (pf_3) = 2-Week Meal Plan (Breakfast, Lunch, Dinner)
Product Features 4 (pf_4) = Complete Keto Diet Food List


In [8]:
p.variants.first().get_absolute_url()

'/en/products/essential-keto-cookbook-121/'

In [379]:
av = [a.values.first() for a in p.attributes.all() if a.attribute.slug == 'external_url'][0]

In [381]:
av.__dict__

{'_state': <django.db.models.base.ModelState at 0x7fa91a162d90>,
 'id': 89,
 'sort_order': 3,
 'name': 'http://localhost/banana-juice',
 'value': '',
 'slug': 'httplocalhostbanana-juice',
 'attribute_id': 27}

In [382]:
av.name = 'http://localhost/banana-juice/'

In [383]:
av.save()

In [344]:
def get_first_attr_value(product, slug):
    return product.attributes.get(assignment__attribute__slug=slug).assignment.attribute.values.first().name

In [356]:
apa = p.attributes.get(assignment__attribute__slug='external_url')

In [364]:
apa.assignment.attribute.values.all()

<QuerySet [<AttributeValue: foo>, <AttributeValue: http://localhost/carrot-juice/>]>

In [354]:
get_first_attr_value(p, 'external_url')

'foo'

In [339]:
[a.assignment.attribute.values.first().name for a in p.attributes.filter(assignment__attribute__slug='external_url')]

['foo']

In [319]:
[(a.attribute.slug, a.assignment.attribute.values.all()) for a in p.attributes.all()]

[('flavor',
  <QuerySet [<AttributeValue: Orange>, <AttributeValue: Banana>, <AttributeValue: Bean>, <AttributeValue: Carrot>, <AttributeValue: Sprouty>, <AttributeValue: Salty, like the tears of your enemy>, <AttributeValue: Pineapple>, <AttributeValue: Coconut>, <AttributeValue: Apple>]>),
 ('funnel_order',
  <QuerySet [<AttributeValue: Primary>, <AttributeValue: Secondary>, <AttributeValue: Tertiary>]>),
 ('external_url', <QuerySet [<AttributeValue: foo>]>)]

In [280]:
pv = ProductVariant.objects.get(id=206); pv.product

<saleor.product.models.Product(pk=73, name='Carrot Juice')>

In [284]:
Collection

saleor.product.models.Collection

### Users and meta data

In [150]:
user = User.objects.get(email="chris@nourishbalancethrive.com")

In [151]:
user.private_meta

{'payment-gateways': {'DUMMY': {'customer_id': None},
  'STRIPE': {'customer_id': 'cus_GNciwmWqsAiSWp'},
  'BRAINTREE': {'customer_id': '254012641'}}}

In [160]:
import stripe
stripe.api_key = 'sk_test_ZPbcFgfw0fJwDT5S3iTbKtvZ00PKAqC1JC'

In [161]:
customer_id = fetch_customer_id(user, "Stripe")

In [162]:
stripe.Customer.retrieve(customer_id)

INFO stripe message='Request to Stripe api' method=get path=https://api.stripe.com/v1/customers/cus_GNciwmWqsAiSWp [PID:26089:MainThread]
INFO stripe message='Stripe API response' path=https://api.stripe.com/v1/customers/cus_GNciwmWqsAiSWp response_code=200 [PID:26089:MainThread]


<Customer customer id=cus_GNciwmWqsAiSWp at 0x7fa91ae0bf50> JSON: {
  "address": null,
  "balance": 0,
  "created": 1576632998,
  "currency": null,
  "default_source": null,
  "delinquent": false,
  "description": null,
  "discount": null,
  "email": "chris@nourishbalancethrive.com",
  "id": "cus_GNciwmWqsAiSWp",
  "invoice_prefix": "DAA454C8",
  "invoice_settings": {
    "custom_fields": null,
    "default_payment_method": null,
    "footer": null
  },
  "livemode": false,
  "metadata": {},
  "name": null,
  "object": "customer",
  "phone": null,
  "preferred_locales": [],
  "shipping": null,
  "sources": {
    "data": [],
    "has_more": false,
    "object": "list",
    "total_count": 0,
    "url": "/v1/customers/cus_GNciwmWqsAiSWp/sources"
  },
  "subscriptions": {
    "data": [],
    "has_more": false,
    "object": "list",
    "total_count": 0,
    "url": "/v1/customers/cus_GNciwmWqsAiSWp/subscriptions"
  },
  "tax_exempt": "none",
  "tax_ids": {
    "data": [],
    "has_more": fa

In [164]:
payment_method = stripe.PaymentMethod.retrieve(
  "pm_1Fr51ODGt0TthQJw4hzissd3",
)

INFO stripe message='Request to Stripe api' method=get path=https://api.stripe.com/v1/payment_methods/pm_1Fr51ODGt0TthQJw4hzissd3 [PID:26089:MainThread]
INFO stripe message='Stripe API response' path=https://api.stripe.com/v1/payment_methods/pm_1Fr51ODGt0TthQJw4hzissd3 response_code=200 [PID:26089:MainThread]


In [166]:
payment_method.customer

'cus_GNciwmWqsAiSWp'

In [143]:
store_customer_id(user, "Stripe", None)

In [138]:
user

<User: chris@nourishbalancethrive.com>

### Checkout

In [490]:
checkout = Checkout.objects.first(); checkout

Checkout(quantity=1)

In [491]:
checkout.__dict__

{'_state': <django.db.models.base.ModelState at 0x7fa91aa4ca50>,
 'private_meta': {},
 'meta': {},
 'created': datetime.datetime(2020, 1, 2, 2, 38, 50, 338857, tzinfo=<UTC>),
 'last_change': datetime.datetime(2020, 1, 2, 2, 38, 50, 338897, tzinfo=<UTC>),
 'user_id': None,
 'email': 'chris@nourishbalancethrive.com',
 'token': UUID('bfb51d41-c13d-4a9b-b204-169010208ccc'),
 'quantity': 1,
 'billing_address_id': None,
 'shipping_address_id': 2216,
 'shipping_method_id': 13,
 'note': '',
 'currency': 'USD',
 'discount_amount': Decimal('0.00'),
 'discount_name': None,
 'translated_discount_name': None,
 'voucher_code': None}

In [492]:
checkout.delete()

(2,
 {'checkout.Checkout_gift_cards': 0,
  'checkout.CheckoutLine': 1,
  'checkout.Checkout': 1})

In [9]:
o1 = Order.objects.first(); o1.token, o1

('07720ff7-2a3e-445b-9ea1-eeb79b50aa5f', <Order #280>)

In [10]:
o1.lines.count()

3

In [15]:
o1.store_meta('funnel', 'events', {'send_order_confirmation': True})

In [17]:
o1.get_meta('funnel', 'events')

{'send_order_confirmation': True}

In [16]:
o1.meta

{'funnel': {'events': {'send_order_confirmation': True}}}

In [11]:
o1.__dict__

{'_state': <django.db.models.base.ModelState at 0x7fed2aad0cd0>,
 'id': 280,
 'private_meta': {},
 'meta': {},
 'created': datetime.datetime(2020, 1, 15, 16, 40, 25, 684850, tzinfo=<UTC>),
 'status': 'unfulfilled',
 'user_id': 1,
 'language_code': 'en',
 'tracking_client_id': '5dce0806-ed60-58c7-bd47-0919e2bdd2bb',
 'billing_address_id': 2801,
 'shipping_address_id': 653,
 'user_email': 'chris@nourishbalancethrive.com',
 'currency': 'USD',
 'shipping_method_id': 13,
 'shipping_method_name': 'UPS',
 'shipping_price_net_amount': Decimal('10.90'),
 'shipping_price_gross_amount': Decimal('10.90'),
 'token': '07720ff7-2a3e-445b-9ea1-eeb79b50aa5f',
 'checkout_token': '4197b2c6-0c3c-461f-921e-bcd2566774ac',
 'total_net_amount': Decimal('69.67'),
 'total_gross_amount': Decimal('69.67'),
 'voucher_id': None,
 'discount_amount': Decimal('0.00'),
 'discount_name': '',
 'translated_discount_name': '',
 'display_gross_prices': True,
 'customer_note': '',
 'weight': Mass(kg=1.0)}

In [495]:
[p.__dict__ for p in o1.payments.all()]

[{'_state': <django.db.models.base.ModelState at 0x7fa91ad86090>,
  'id': 366,
  'gateway': 'Braintree',
  'is_active': True,
  'created': datetime.datetime(2020, 1, 2, 2, 39, 3, 820951, tzinfo=<UTC>),
  'modified': datetime.datetime(2020, 1, 2, 2, 39, 3, 842982, tzinfo=<UTC>),
  'charge_status': 'not-charged',
  'token': '',
  'total': Decimal('10.90'),
  'captured_amount': Decimal('0.00'),
  'currency': 'USD',
  'checkout_id': None,
  'order_id': 266,
  'billing_email': '',
  'billing_first_name': '',
  'billing_last_name': '',
  'billing_company_name': '',
  'billing_address_1': '',
  'billing_address_2': '',
  'billing_city': '',
  'billing_city_area': '',
  'billing_postal_code': '',
  'billing_country_code': '',
  'billing_country_area': '',
  'cc_first_digits': '',
  'cc_last_digits': '',
  'cc_brand': '',
  'cc_exp_month': None,
  'cc_exp_year': None,
  'customer_ip_address': '10.0.2.2',
  'extra_data': "{'customer_user_agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_1) A

In [496]:
[[t.delete() for t in p.transactions.all()] for p in o1.payments.all()]

[[]]

In [497]:
[p.delete() for p in o1.payments.all()]

[(1, {'payment.Payment': 1})]

In [498]:
o1.delete()

(3,
 {'order.Order_gift_cards': 0,
  'product.DigitalContentUrl': 0,
  'order.FulfillmentLine': 0,
  'order.OrderEvent': 1,
  'order.OrderLine': 1,
  'order.Order': 1})

In [399]:
o1.user.private_meta

{'payment-gateways': {'DUMMY': {'customer_id': None},
  'STRIPE': {'customer_id': 'cus_GNciwmWqsAiSWp'},
  'BRAINTREE': {'customer_id': '254012641'}}}

In [69]:
o1.is_fully_paid()

False

In [70]:
o1.billing_address

In [71]:
o1.shipping_address.as_data()

AttributeError: 'NoneType' object has no attribute 'as_data'

In [49]:
o1.shipping_method

In [32]:
d

{'csrfmiddlewaretoken': ['cxHqQBwfi6Dt6hhXpyHEdsoWdAY5AxMk7lmMFLYQiWcYpQOotkhQEscuHb3HyIda',
  'cxHqQBwfi6Dt6hhXpyHEdsoWdAY5AxMk7lmMFLYQiWcYpQOotkhQEscuHb3HyIda'],
 'address': ['new_address'],
 'first_name': ['Christopher'],
 'last_name': ['Kelly'],
 'company_name': ['Nourish Balance Thrive'],
 'street_address_1': ['471 Country Estates Ter'],
 'street_address_2': [''],
 'city': ['Santa Cruz'],
 'country_area': ['CA'],
 'postal_code': ['95060'],
 'city_area': [''],
 'country': ['US'],
 'phone_1': [''],
 'preview': [''],
 'shipping_method': ['13'],
 'cardholder-name': ['Christopher Kelly'],
 'payment_method_id': ['pm_1FmLG5DGt0TthQJwmdlFHvWr']}

In [29]:
gettataaao1.redirect

AttributeError: 'Order' object has no attribute 'redirect'

In [22]:
o1.billing_address

In [72]:
o1.delete()

(3,
 {'order.Order_gift_cards': 0,
  'product.DigitalContentUrl': 0,
  'order.FulfillmentLine': 0,
  'order.OrderEvent': 1,
  'order.OrderLine': 1,
  'order.Order': 1})

In [26]:
user = User.objects.get(email='chris@nourishbalancethrive.com')

In [27]:
user.addresses.all()

<AddressQueryset [<Address: Nourish Balance Thrive - Christopher Kelly>]>

In [17]:
[addr.delete() for addr in user.addresses.all()]

[(2, {'account.User_addresses': 1, 'account.Address': 1}),
 (2, {'account.User_addresses': 1, 'account.Address': 1}),
 (2, {'account.User_addresses': 1, 'account.Address': 1}),
 (2, {'account.User_addresses': 1, 'account.Address': 1})]

In [90]:
o1.currency

'USD'

In [98]:
print(o1.total_gross.amount)

41.50


In [17]:
o1.token

'374ca1c7-5aa8-4b44-9d9c-f1147f818e33'

In [15]:
upsell_order(o1, checkout, "")

<Order #111>

In [16]:
o1.payments.first().gateway

'Braintree'

In [17]:
len(o1.payments.all())

1

In [16]:
%pdb

Automatic pdb calling has been turned OFF


In [18]:
payment = o1.payments.first()
extra_data = {f"upsell_payment_number": f'{len(o1.payments.all())}'}
payment_ = create_payment(
      gateway=payment.gateway,
      currency=o1.total.gross.currency,
      email=o1.user_email,
      billing_address=o1.billing_address,
      customer_ip_address=payment.customer_ip_address,
      total=abs(o1.total_balance.amount),
      order=o1,
      extra_data=extra_data,
  )

In [19]:
extra_data

{'upsell_payment_number': '1'}

In [20]:
payment_, payment_.pk

(Payment(gateway=Braintree, is_active=True, created=2019-11-22 21:12:21.847482+00:00, charge_status=not-charged),
 150)

In [21]:
transaction = payment.transactions.first()
token = transaction.gateway_response.get('payment_method', '')
customer_id = transaction.customer_id

In [23]:
customer_id, token

('896095806', '')

In [24]:
transaction = process_payment(payment_, token, store_source=True, customer_id=customer_id)

process_payment: token:  store_source: True
process_payment: reuse_source: True
braintree.authorize: payment_information: PaymentData(amount=Decimal('7.00'), currency='USD', billing=AddressData(first_name='Julia', last_name='Kelly', company_name='Nourish Balance Thrive', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), shipping=AddressData(first_name='Julia', last_name='Kelly', company_name='Nourish Balance Thrive', street_address_1='471 Country Estates Ter', street_address_2='', city='SANTA CRUZ', city_area='', postal_code='95060', country='US', country_area='CA', phone='+14157069979'), order_id=111, customer_ip_address='10.0.2.2', customer_email='chris@nourishbalancethrive.com', token='', customer_id='896095806', reuse_source=True)
authorize: gateway_response: {'transaction_id': 'ah26wx02', 'currency': 'USD', 'amount': Decimal('7.00'), 'credit_card': {'token':

PaymentError: Transaction was unsuccessful

In [ ]:
GatewayResponse(is_success=True, action_required=False, kind='capture', amount=Decimal('13.90'), currency='USD', transaction_id='4j7vnf6v', error='', cust
omer_id='409815689', card_info=None, raw_response={'transaction_id': '4j7vnf6v', 'currency': 'USD', 'amount': Decimal('13.90'), 'credit_card': {'token': '43thd9', 'bin': None, 'last
_4': None, 'card_type': None, 'expiration_month': '', 'expiration_year': '', 'customer_location': None, 'cardholder_name': None, 'image_url': 'https://assets.braintreegateway.com/pa
yment_method_logo/unknown.png?environment=sandbox', 'prepaid': 'Unknown', 'healthcare': 'Unknown', 'debit': 'Unknown', 'durbin_regulated': 'Unknown', 'commercial': 'Unknown', 'payro
ll': 'Unknown', 'issuing_bank': 'Unknown', 'country_of_issuance': 'Unknown', 'product_id': 'Unknown', 'global_id': 'cGF5bWVudG1ldGhvZF9jY180M3RoZDk', 'account_type': None, 'unique_n
umber_identifier': None, 'venmo_sdk': False}, 'customer_id': '409815689', 'errors': []}), error: None

In [17]:
transaction.pk

102

In [214]:
19+10.9

29.9

In [217]:
o1.save()

In [175]:
o1.payments.all()

<QuerySet [Payment(gateway=Dummy, is_active=True, created=2019-11-14 15:01:58.499543+00:00, charge_status=fully-charged), Payment(gateway=Dummy, is_active=True, created=2019-11-14 17:05:32.774685+00:00, charge_status=fully-charged)]>

In [144]:
payment2.transactions.all()

<QuerySet [Transaction(type=capture, is_success=True, created=2019-11-13 16:45:48.503291+00:00)]>

In [152]:
o1.payments.all()

<QuerySet [Payment(gateway=Dummy, is_active=True, created=2019-11-13 16:42:32.471117+00:00, charge_status=fully-charged), Payment(gateway=Dummy, is_active=True, created=2019-11-13 16:45:38.533943+00:00, charge_status=fully-charged)]>

In [145]:
payment.transactions.add(transaction)

In [146]:
payment.save()

In [147]:
payment.total

Decimal('13.90')

In [150]:
[t.amount for t in payment.transactions.all()]

[Decimal('13.90'), Decimal('-7.00'), Decimal('7.00'), Decimal('0.00')]

I think the code above works, but the methods in `Order` and maybe `Payment` don't work for multiple payments. Add new methods and adjust dashboard templates accordingly.